In [494]:
import numpy as np
import pandas as pd
import networkx as nx
from collections import defaultdict
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MultiLabelBinarizer
from sklearn.metrics import accuracy_score, classification_report

In [495]:
# Define symptom weights for each disease
symptom_weights = {
    "Malaria": {
        "Fever": 1.0, "Chills": 0.95, "Sweating": 0.9, "Headache": 0.8, "Fatigue": 0.75,
        "Muscle Pain": 0.7, "Nausea": 0.6, "Vomiting": 0.55, "Joint Pain": 0.5, "Diarrhea": 0.4
    },
    "Typhoid Fever": {
        "High Fever": 1.0, "Abdominal Pain": 0.9, "Weakness": 0.85, "Headache": 0.8, "Loss of Appetite": 0.75,
        "Fatigue": 0.7, "Chills": 0.65, "Constipation": 0.6, "Diarrhea": 0.55, "Rash": 0.5
    },
    "Cholera": {
        "Severe Diarrhea": 1.0, "Dehydration": 0.95, "Vomiting": 0.85, "Thirst": 0.8, "Low Blood Pressure": 0.75,
        "Rapid Heart Rate": 0.7, "Muscle Cramps": 0.65, "Dry Mouth": 0.6, "Restlessness": 0.5, "Leg Cramps": 0.4
    },
    "Pneumonia": {
        "Cough": 1.0, "Shortness of Breath": 0.95, "Fever": 0.9, "Chest Pain": 0.85, "Fatigue": 0.7,
        "Wheezing": 0.65, "Chills": 0.6, "Sweating": 0.5, "Loss of Appetite": 0.45, "Headache": 0.4
    },
    "Tuberculosis": {
        "Chronic Cough": 1.0, "Coughing Blood": 0.95, "Night Sweats": 0.9, "Weight Loss": 0.85, "Fever": 0.8,
        "Fatigue": 0.75, "Chest Pain": 0.7, "Shortness of Breath": 0.65, "Loss of Appetite": 0.6, "Weakness": 0.55
    },
    "Diabetes": {
        "Increased Thirst": 1.0, "Frequent Urination": 0.95, "Unexplained Weight Loss": 0.9, "Fatigue": 0.85, "Blurred Vision": 0.8,
        "Slow Wound Healing": 0.75, "Numbness": 0.7, "Tingling": 0.65, "Dry Skin": 0.6, "Irritability": 0.5
    },
    "Hypertension": {
        "Headaches": 0.8, "Dizziness": 0.75, "Chest Pain": 0.9, "Shortness of Breath": 0.85, "Nosebleeds": 0.7,
        "Blurred Vision": 0.75, "Fatigue": 0.65, "Confusion": 0.6, "Anxiety": 0.5, "Nausea": 0.45
    },
    "Dengue Fever": {
        "High Fever": 1.0, "Severe Headache": 0.95, "Joint and Muscle Pain": 0.9, "Rash": 0.85, "Eye Pain": 0.8,
        "Fatigue": 0.75, "Bleeding": 0.7, "Nausea": 0.65, "Vomiting": 0.6, "Swollen Glands": 0.55
    },
    "Hepatitis B": {
        "Jaundice": 1.0, "Dark Urine": 0.95, "Fatigue": 0.9, "Abdominal Pain": 0.85, "Loss of Appetite": 0.8,
        "Nausea": 0.75, "Vomiting": 0.7, "Joint Pain": 0.6, "Fever": 0.55, "Swelling": 0.5
    },
    "Meningitis": {
        "Stiff Neck": 1.0, "Severe Headache": 0.95, "Fever": 0.9, "Sensitivity to Light": 0.85, "Confusion": 0.8,
        "Vomiting": 0.75, "Seizures": 0.7, "Sleepiness": 0.65, "Skin Rash": 0.6, "Difficulty Concentrating": 0.55
    },
    "Asthma": {
        "Shortness of Breath": 1.0, "Wheezing": 0.95, "Chest Tightness": 0.9, "Coughing": 0.85, "Difficulty Sleeping": 0.7,
        "Rapid Breathing": 0.65, "Anxiety": 0.6, "Fatigue": 0.55, "Bluish Lips": 0.5, "Sweating": 0.45
    },
    "Bronchitis": {
        "Cough": 1.0, "Chest Discomfort": 0.9, "Shortness of Breath": 0.85, "Wheezing": 0.8, "Fatigue": 0.7,
        "Fever": 0.65, "Chills": 0.6, "Sore Throat": 0.55, "Muscle Aches": 0.5, "Runny Nose": 0.45
    },
    "Influenza": {
        "Fever": 1.0, "Body Aches": 0.95, "Fatigue": 0.9, "Cough": 0.85, "Headache": 0.8,
        "Chills": 0.75, "Sore Throat": 0.7, "Runny Nose": 0.65, "Sneezing": 0.6, "Nausea": 0.5
    },
    "HIV/AIDS": {
        "Weight Loss": 0.9, "Fatigue": 0.85, "Frequent Infections": 1.0, "Swollen Lymph Nodes": 0.95, "Night Sweats": 0.8,
        "Persistent Cough": 0.75, "Diarrhea": 0.7, "Skin Rashes": 0.65, "Mouth Ulcers": 0.6, "Joint Pain": 0.55
    },
    "Ebola": {
        "High Fever": 1.0, "Bleeding": 0.95, "Severe Headache": 0.9, "Vomiting": 0.85, "Muscle Pain": 0.8,
        "Weakness": 0.75, "Diarrhea": 0.7, "Sore Throat": 0.65, "Rash": 0.6, "Confusion": 0.55
    },
    "Yellow Fever": {
        "Fever": 0.95, "Jaundice": 1.0, "Bleeding": 0.9, "Headache": 0.85, "Muscle Pain": 0.8,
        "Back Pain": 0.75, "Nausea": 0.7, "Vomiting": 0.65, "Fatigue": 0.6, "Organ Failure": 0.55
    },
    "Leptospirosis": {
        "Fever": 1.0, "Jaundice": 0.95, "Muscle Pain": 0.9, "Headache": 0.85, "Chills": 0.8,
        "Eye Redness": 0.75, "Kidney Damage": 0.7, "Nausea": 0.65, "Vomiting": 0.6, "Rash": 0.55
    },
    "Schistosomiasis": {
        "Blood in Urine": 1.0, "Abdominal Pain": 0.95, "Fever": 0.9, "Diarrhea": 0.85, "Bloody Stool": 0.8,
        "Fatigue": 0.75, "Cough": 0.7, "Nausea": 0.65, "Vomiting": 0.6, "Rash": 0.55
    },
    "Lassa Fever": {
        "Fever": 1.0, "Bleeding": 0.95, "Weakness": 0.9, "Vomiting": 0.85, "Chest Pain": 0.8,
        "Sore Throat": 0.75, "Hearing Loss": 0.7, "Seizures": 0.65, "Swollen Face": 0.6, "Shock": 0.55
    },
    "Tetanus": {
        "Muscle Stiffness": 1.0, "Lockjaw": 0.95, "Difficulty Swallowing": 0.9, "Seizures": 0.85, "High Blood Pressure": 0.8,
        "Rapid Heart Rate": 0.75, "Fever": 0.7, "Sweating": 0.65, "Irritability": 0.6, "Headache": 0.55
    },
    "Measles": {
        "High Fever": 0.95, "Skin Rash": 1.0, "Cough": 0.9, "Red Eyes": 0.85, "Runny Nose": 0.8,
        "Sore Throat": 0.75, "Sensitivity to Light": 0.7, "Fatigue": 0.65, "Loss of Appetite": 0.6, "Muscle Aches": 0.55
    },
    "Rubella": {
        "Rash": 1.0, "Mild Fever": 0.95, "Swollen Lymph Nodes": 0.9, "Joint Pain": 0.85, "Red Eyes": 0.8,
        "Headache": 0.75, "Sore Throat": 0.7, "Muscle Pain": 0.65, "Fatigue": 0.6, "Runny Nose": 0.55
    },
    "Poliomyelitis": {
        "Paralysis": 1.0, "Muscle Weakness": 0.95, "Fever": 0.9, "Headache": 0.85, "Stiff Neck": 0.8,
        "Sore Throat": 0.75, "Fatigue": 0.7, "Painful Limbs": 0.65, "Difficulty Breathing": 0.6, "Vomiting": 0.55
    },
    "Rabies": {
        "Fear of Water": 1.0, "Excess Salivation": 0.95, "Muscle Spasms": 0.9, "Confusion": 0.85, "Paralysis": 0.8,
        "Anxiety": 0.75, "Difficulty Swallowing": 0.7, "Fever": 0.65, "Headache": 0.6, "Insomnia": 0.55
    },
    "Whooping Cough": {
        "Severe Cough": 1.0, "Choking": 0.95, "Difficulty Breathing": 0.9, "Vomiting": 0.85, "Bluish Lips": 0.8,
        "Fatigue": 0.75, "Runny Nose": 0.7, "Mild Fever": 0.65, "Sneezing": 0.6, "Sweating": 0.55
    },
    "Gonorrhea": {
        "Painful Urination": 0.95, "Pus-like Discharge": 1.0, "Swollen Testicles": 0.9, "Painful Intercourse": 0.85, "Lower Abdominal Pain": 0.8,
        "Irregular Bleeding": 0.75, "Rectal Pain": 0.7, "Sore Throat": 0.65, "Fever": 0.6, "Joint Pain": 0.55
    },
    "Syphilis": {
        "Sores": 1.0, "Rashes": 0.95, "Swollen Lymph Nodes": 0.9, "Fever": 0.85, "Fatigue": 0.8,
        "Headaches": 0.75, "Joint Pain": 0.7, "Weight Loss": 0.65, "Hair Loss": 0.6, "Vision Problems": 0.55
    },
    "Trichomoniasis": {
        "Foul-Smelling Discharge": 1.0, "Itching": 0.95, "Burning": 0.9, "Painful Urination": 0.85, "Discomfort": 0.8,
        "Painful Intercourse": 0.75, "Redness": 0.7, "Swelling": 0.65, "Frequent Urination": 0.6, "Abdominal Pain": 0.55
    },
    "Lymphatic Filariasis": {
        "Swollen Limbs": 1.0, "Thickened Skin": 0.95, "Swollen Lymph Nodes": 0.9, "Fever": 0.85, "Fatigue": 0.8,
        "Chills": 0.75, "Muscle Pain": 0.7, "Headache": 0.65, "Nausea": 0.6, "Abdominal Pain": 0.55
    },
    "Onchocerciasis": {
        "Itching": 0.95, "Nodules under Skin": 0.9, "Blindness": 1.0, "Skin Rash": 0.85, "Swollen Lymph Nodes": 0.8,
        "Fever": 0.75, "Weight Loss": 0.7, "Fatigue": 0.65, "Headache": 0.6, "Joint Pain": 0.55
    },
    "Trypanosomiasis": {
        "Sleep Disturbances": 1.0, "Fever": 0.95, "Confusion": 0.9, "Swollen Lymph Nodes": 0.85, "Headache": 0.8,
        "Fatigue": 0.75, "Joint Pain": 0.7, "Itching": 0.65, "Weight Loss": 0.6, "Muscle Pain": 0.55
    },
    "Bilharzia": {
        "Blood in Urine": 1.0, "Abdominal Pain": 0.95, "Swollen Abdomen": 0.9, "Diarrhea": 0.85, "Fatigue": 0.8,
        "Fever": 0.75, "Weight Loss": 0.7, "Cough": 0.65, "Joint Pain": 0.6, "Itching": 0.55
    },
    "Guinea Worm Disease": {
        "Painful Blisters": 1.0, "Ulcers": 0.95, "Swelling": 0.9, "Fever": 0.85, "Itching": 0.8,
        "Fatigue": 0.75, "Muscle Pain": 0.7, "Nausea": 0.65, "Vomiting": 0.6, "Diarrhea": 0.55
    },
    "Buruli Ulcer": {
        "Skin Ulcers": 1.0, "Painless Nodules": 0.95, "Swelling": 0.9, "Redness": 0.85, "Fatigue": 0.8,
        "Fever": 0.75, "Weight Loss": 0.7, "Swollen Lymph Nodes": 0.65, "Itching": 0.6, "Muscle Pain": 0.55
    },
    "Trachoma": {
        "Eye Pain": 0.95, "Blurred Vision": 1.0, "Sensitivity to Light": 0.9, "Swollen Eyelids": 0.85, "Redness": 0.8,
        "Discharge": 0.75, "Itching": 0.7, "Headache": 0.65, "Fever": 0.6, "Fatigue": 0.55
    },
    "Scabies": {
        "Itching": 1.0, "Rash": 0.95, "Sores": 0.9, "Thickened Skin": 0.85, "Redness": 0.8,
        "Swelling": 0.75, "Sleep Disturbances": 0.7, "Irritability": 0.65, "Fever": 0.6, "Fatigue": 0.55
    },
    "Ringworm": {
        "Itching": 0.95, "Red Patches": 1.0, "Scaling": 0.9, "Rash": 0.85, "Hair Loss": 0.8,
        "Blistering": 0.75, "Discomfort": 0.7, "Swelling": 0.65, "Fatigue": 0.6, "Fever": 0.55
    },
    "Hookworm": {
        "Anemia": 1.0, "Fatigue": 0.95, "Abdominal Pain": 0.9, "Weight Loss": 0.85, "Diarrhea": 0.8,
        "Nausea": 0.75, "Itching": 0.7, "Cough": 0.65, "Vomiting": 0.6, "Fever": 0.55
    },
    "Ascariasis": {
        "Abdominal Pain": 1.0, "Cough": 0.95, "Wheezing": 0.9, "Diarrhea": 0.85, "Nausea": 0.8,
        "Vomiting": 0.75, "Weight Loss": 0.7, "Fatigue": 0.65, "Fever": 0.6, "Itching": 0.55
    },
    "Giardiasis": {
        "Diarrhea": 1.0, "Abdominal Pain": 0.95, "Nausea": 0.9, "Bloating": 0.85, "Dehydration": 0.8,
        "Fatigue": 0.75, "Weight Loss": 0.7, "Vomiting": 0.65, "Fever": 0.6, "Headache": 0.55
    },
    "Amebiasis": {
        "Bloody Stool": 1.0, "Diarrhea": 0.95, "Abdominal Pain": 0.9, "Fatigue": 0.85, "Weight Loss": 0.8,
        "Fever": 0.75, "Nausea": 0.7, "Vomiting": 0.65, "Dehydration": 0.6, "Headache": 0.55
    },
    "Cryptosporidiosis": {
        "Diarrhea": 1.0, "Dehydration": 0.95, "Abdominal Pain": 0.9, "Nausea": 0.85, "Weight Loss": 0.8,
        "Fatigue": 0.75, "Vomiting": 0.7, "Fever": 0.65, "Headache": 0.6, "Muscle Pain": 0.55
    },
    "Toxoplasmosis": {
        "Swollen Lymph Nodes": 1.0, "Fever": 0.95, "Fatigue": 0.9, "Headache": 0.85, "Confusion": 0.8,
        "Seizures": 0.75, "Blurred Vision": 0.7, "Muscle Pain": 0.65, "Sore Throat": 0.6, "Weight Loss": 0.55
    },
    "Leishmaniasis": {
        "Skin Sores": 1.0, "Fever": 0.95, "Weight Loss": 0.9, "Swollen Spleen": 0.85, "Fatigue": 0.8,
        "Anemia": 0.75, "Swollen Lymph Nodes": 0.7, "Nosebleeds": 0.65, "Cough": 0.6, "Muscle Pain": 0.55
    },
    "Dracunculiasis": {
        "Painful Blisters": 1.0, "Ulcers": 0.95, "Swelling": 0.9, "Itching": 0.85, "Fever": 0.8,
        "Fatigue": 0.75, "Muscle Pain": 0.7, "Nausea": 0.65, "Vomiting": 0.6, "Diarrhea": 0.55
    },
    "African Trypanosomiasis": {
        "Sleep Disturbances": 1.0, "Fever": 0.95, "Confusion": 0.9, "Swollen Lymph Nodes": 0.85, "Headache": 0.8,
        "Fatigue": 0.75, "Joint Pain": 0.7, "Itching": 0.65, "Weight Loss": 0.6, "Muscle Pain": 0.55
    },
    "River Blindness": {
        "Blindness": 1.0, "Itching": 0.95, "Nodules under Skin": 0.9, "Skin Rash": 0.85, "Swollen Lymph Nodes": 0.8,
        "Fever": 0.75, "Weight Loss": 0.7, "Fatigue": 0.65, "Headache": 0.6, "Joint Pain": 0.55
    },
    "Sleeping Sickness": {
        "Sleep Disturbances": 1.0, "Confusion": 0.95, "Fever": 0.9, "Swollen Lymph Nodes": 0.85, "Headache": 0.8,
        "Fatigue": 0.75, "Joint Pain": 0.7, "Itching": 0.65, "Weight Loss": 0.6, "Muscle Pain": 0.55
    },
    "Leprosy": {
        "Skin Lesions": 1.0, "Numbness": 0.95, "Muscle Weakness": 0.9, "Deformities": 0.85, "Blindness": 0.8,
        "Joint Pain": 0.75, "Swollen Lymph Nodes": 0.7, "Fatigue": 0.65, "Weight Loss": 0.6, "Fever": 0.55
    },
    "Yaws": {
        "Skin Sores": 1.0, "Joint Pain": 0.95, "Fever": 0.9, "Swollen Lymph Nodes": 0.85, "Rash": 0.8,
        "Fatigue": 0.75, "Weight Loss": 0.7, "Itching": 0.65, "Headache": 0.6, "Muscle Pain": 0.55
    }
}

In [496]:
# Function to generate realistic samples
def generate_realistic_samples(disease, num_samples=10):
    samples = []
    weights = symptom_weights[disease]
    core_symptoms = list(weights.keys())[:2]  # First 2 symptoms are core
    secondary_symptoms = list(weights.keys())[2:]

    # Normalize probabilities for secondary symptoms
    secondary_weights = np.array([weights[s] for s in secondary_symptoms])
    secondary_weights /= secondary_weights.sum()

    for _ in range(num_samples):
        # Always include core symptoms
        selected = list(core_symptoms)
        # Add 2-5 secondary symptoms
        num_secondary = np.random.randint(2, 6)
        secondary_selected = np.random.choice(
            secondary_symptoms,
            size=num_secondary,
            p=secondary_weights,
            replace=False
        )
        selected.extend(secondary_selected)
        samples.append([disease] + selected)
    return samples

In [497]:
# Create new augmented dataset
new_augmented = []
for disease in symptom_weights.keys():
    samples = generate_realistic_samples(disease, num_samples=50)  # Increased samples
    new_augmented.extend(samples)

In [498]:
# Save to new dataframe
max_symptoms = max(len(sample) - 1 for sample in new_augmented)
new_df = pd.DataFrame(new_augmented, columns=["Disease"] + [f"Symptom {i}" for i in range(1, max_symptoms + 1)])

In [499]:
# Load enhanced dataset
df = new_df

In [500]:
# Preprocess data
symptoms_list = []
disease_list = []
for _, row in df.iterrows():
    symptoms = [s for s in row[1:] if pd.notna(s)]
    symptoms_list.append(symptoms)
    disease_list.append(row['Disease'])

In [501]:
# Create binary features
mlb = MultiLabelBinarizer()
X = mlb.fit_transform(symptoms_list)
y = disease_list

In [502]:
# Encode labels
le = LabelEncoder()
y_encoded = le.fit_transform(y)

In [503]:
# Split into train/test sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y_encoded,
    test_size=0.2,
    random_state=42,
    stratify=y_encoded
)

In [504]:
# Train model with class weights
model = RandomForestClassifier(
    n_estimators=200,
    class_weight="balanced",  # Handle imbalanced classes
    random_state=42
)
model.fit(X_train, y_train)

RandomForestClassifier(class_weight='balanced', n_estimators=200,
                       random_state=42)

In [505]:
# Evaluate
y_pred = model.predict(X_test)
print(f"Accuracy: {accuracy_score(y_test, y_pred):.2f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=le.classes_))

Accuracy: 0.93

Classification Report:
                         precision    recall  f1-score   support

African Trypanosomiasis       0.58      0.70      0.64        10
              Amebiasis       1.00      1.00      1.00        10
             Ascariasis       1.00      1.00      1.00        10
                 Asthma       1.00      1.00      1.00        10
              Bilharzia       1.00      1.00      1.00        10
             Bronchitis       1.00      1.00      1.00        10
           Buruli Ulcer       1.00      1.00      1.00        10
                Cholera       1.00      1.00      1.00        10
      Cryptosporidiosis       1.00      0.70      0.82        10
           Dengue Fever       1.00      1.00      1.00        10
               Diabetes       1.00      1.00      1.00        10
         Dracunculiasis       0.46      0.60      0.52        10
                  Ebola       1.00      1.00      1.00        10
             Giardiasis       0.77      1.00      

In [506]:
# Function to predict diseases
def predict_disease(symptoms, top_n=3):
    input_vector = np.zeros(len(mlb.classes_))
    for symptom in symptoms:
        if symptom in mlb.classes_:
            idx = list(mlb.classes_).index(symptom)
            input_vector[idx] = 1

    # Get probabilities
    probabilities = model.predict_proba([input_vector])[0]
    disease_indices = np.argsort(probabilities)[::-1]

    # Get top predictions
    results = []
    for idx in disease_indices[:top_n]:
        disease = le.inverse_transform([idx])[0]
        prob = probabilities[idx]
        results.append({"disease": disease, "probability": round(prob, 2)})

    return results

In [507]:
# Example test
test_symptoms = [ "Fever", "Chills", "Sweating", "Headache", "Fatigue",
        "Muscle Pain", "Nausea", "Vomiting", "Joint Pain", "Diarrhea"]
predictions = predict_disease(test_symptoms)
print("Top Predictions:")
for pred in predictions:
    print(f"{pred['disease']}: {pred['probability']*100}%")

Top Predictions:
Malaria: 98.0%
Leptospirosis: 0.0%
Cryptosporidiosis: 0.0%


In [508]:
# Enhanced symptom relationship calculation
def build_symptom_graph(symptoms_list):
    symptom_graph = nx.Graph()
    co_occurrence = defaultdict(int)
    symptom_freq = defaultdict(int)

    # First pass: count individual frequencies
    for symptoms in symptoms_list:
        for symptom in symptoms:
            symptom_freq[symptom] += 1

    # Second pass: count co-occurrences with weighting
    for symptoms in symptoms_list:
        for i in range(len(symptoms)):
            for j in range(i+1, len(symptoms)):
                pair = tuple(sorted([symptoms[i], symptoms[j]]))
                # Weight by inverse symptom frequency (IDF-like weighting)
                weight = 1/(symptom_freq[symptoms[i]] + symptom_freq[symptoms[j]])
                co_occurrence[pair] += weight

    # Add edges with normalized weights
    max_weight = max(co_occurrence.values(), default=1)
    for (sym1, sym2), weight in co_occurrence.items():
        symptom_graph.add_edge(sym1, sym2, weight=weight/max_weight)

    return symptom_graph

# Replace the old graph creation with:
symptom_graph = build_symptom_graph(symptoms_list)

In [509]:
def get_next_symptom(current_symptoms):
    """Get the most relevant next symptom to ask about"""
    candidates = []

    # Find neighbors of current symptoms in graph
    for sym in current_symptoms:
        for neighbor in symptom_graph.neighbors(sym):
            if neighbor not in current_symptoms:
                weight = symptom_graph[sym][neighbor]['weight']
                candidates.append((neighbor, weight))

    # Return symptom with highest co-occurrence weight
    if candidates:
        return max(candidates, key=lambda x: x[1])[0]
    return None

In [510]:
def interactive_diagnosis():
    print("Welcome to SamaCare Symptom Checker!\n")
    current_symptoms = []
    asked_symptoms = set()

    # Let user start with their main symptom
    first_symptom = input("What's your most noticeable symptom? (e.g. Fever, Headache): ").strip()
    current_symptom = first_symptom if first_symptom in mlb.classes_ else None

    while current_symptom:
        answer = input(f"\nDo you have {current_symptom}? (yes/no/stop): ").lower()

        if answer == 'stop':
            break

        asked_symptoms.add(current_symptom)

        if answer == 'yes':
            current_symptoms.append(current_symptom)
            print(f"✓ Added {current_symptom}")

            # Show progress
            predictions = predict_disease(current_symptoms)
            print("\nCurrent Possible Diagnoses:")
            for i, pred in enumerate(predictions[:5], 1):
                print(f"{i}. {pred['disease']} ({pred['probability']*100:.1f}%)")

            # Confidence check
            if predictions and predictions[0]['probability'] > 0.9:
                print("\nHigh confidence diagnosis reached!")
                return

        # Get next symptom suggestion (exclude already asked)
        neighbors = []
        for sym in current_symptoms:
            neighbors.extend([
                n for n in symptom_graph.neighbors(sym)
                if n not in asked_symptoms and n not in current_symptoms
            ])

        # Find most relevant next symptom
        if neighbors:
            freq = pd.Series(neighbors).value_counts()
            current_symptom = freq.idxmax()
        else:
            print("\nNo more relevant symptoms to check.")
            break

    # Final prediction
    if current_symptoms:
        predictions = predict_disease(current_symptoms)
        print("\nFinal Diagnosis Possibilities:")
        for i, pred in enumerate(predictions[:5], 1):
            print(f"{i}. {pred['disease']} ({pred['probability']*100:.1f}%)")
        return predictions

    print("\nInsufficient symptoms for diagnosis. Please consult a doctor.")
    return []

In [511]:
# Run the interactive diagnosis
interactive_diagnosis()

Welcome to SamaCare Symptom Checker!

✓ Added Fever

Current Possible Diagnoses:
1. Toxoplasmosis (18.0%)
2. Malaria (9.0%)
3. Scabies (6.0%)
✓ Added Chills

Current Possible Diagnoses:
1. Malaria (74.0%)
2. Leptospirosis (5.0%)
3. Influenza (4.0%)
✓ Added Fatigue

Current Possible Diagnoses:
1. Malaria (85.0%)
2. Influenza (3.0%)
3. Scabies (2.0%)
✓ Added Sweating

Current Possible Diagnoses:
1. Malaria (96.0%)
2. Influenza (1.0%)
3. River Blindness (0.0%)

High confidence diagnosis reached!
